In [ ]:
import os
import copy
import numpy as np
from CT import construct_CT_object
import os
import json
import glob
import random


: 

In [ ]:

def get_path_structures_DICOM_file(path_CT_DICOM_series):
    filenames = [filename for filename in os.listdir(path_CT_DICOM_series) if filename.startswith('RS')]
    print(filenames)

    assert len(filenames) > 0, "no structures DICOM file found"
    assert len(filenames) <= 1, "more than one structures DICOM files found"

    return os.path.join(path_CT_DICOM_series, filenames[0])



: 

In [ ]:


# root_patient = "/data/oosterhoff/patients/DBP_OP007/"
# name_fixed_CT = 'pCTp0'
# name_moving_CT = 'rCTp12'

# path_fixed_CT = os.path.join(root_patient, name_fixed_CT, '')
# path_moving_CT = os.path.join(root_patient, name_moving_CT, '')


# path_struct_fixed_CT = get_path_structures_DICOM_file(path_fixed_CT)
# path_struct_moving_CT = get_path_structures_DICOM_file(path_moving_CT)



# fixed_CT = construct_CT_object(name_fixed_CT, path_fixed_CT, path_struct_fixed_CT, roi_names = ['External'])
# moving_CT = construct_CT_object(name_moving_CT, path_moving_CT, path_struct_moving_CT, roi_names = ['External'])


: 

In [ ]:
# import matplotlib.pyplot as plt
# import SimpleITK as sitk
# fixed_CT_array = sitk.GetArrayFromImage(fixed_CT.image)
# moving_CT_array = sitk.GetArrayFromImage(moving_CT.image)
# plt.imshow(fixed_CT_array[80])

: 

In [ ]:
class CaseDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)
        
        self.id = self.data['patient_id']
        
        self.examinations_transformations = self.extract_details()



    # def new_transfer_param(self, rigid_matrix):
    #     max_translation = 1.5
    #     num_samples = 5  # Number of random coordinates to generate
    #     final_translation_coordinate = {'x': [], 'y': [], 'z': []}  # Initialize the dictionary with lists
        
    #     matrix = list(rigid_matrix)  # Initialize the matrix outside the loop
    #     for _ in range(num_samples):
    #         for idx, key in zip([3, 7, 11], ['x', 'y', 'z']):
    #             random_translation = (2 * random.random() - 1) * max_translation
    #             matrix[idx] += random_translation * 10  # Apply translation and convert to mm
    #             final_translation_coordinate[key].append(random_translation * 10)  # Store each translation in mm in the list
        
    #     return matrix, final_translation_coordinate

    def new_transfer_param(self, rigid_matrix):
        max_translation = 1.5
        num_samples = 5  # Number of random coordinates to generate
        
        final_translation_coordinate = {}  # Dictionary to hold all options
        new_matrix = {}
        matrix_base = list(rigid_matrix)  # Copy base matrix to modify for each sample
        
        for i in range(1, num_samples + 1):
            opt_key = f'opt{i}'
            final_translation_coordinate[opt_key] = {'x': 0, 'y': 0, 'z': 0}
            current_matrix = matrix_base[:]  # Work with a copy of the base matrix for each sample
            
            new_matrix[opt_key] = current_matrix  # Assign the copy of the base matrix to the option key
            
            for idx, key in zip([3, 7, 11], ['x', 'y', 'z']):
                random_translation = (2 * random.random() - 1) * max_translation
                current_matrix[idx] += random_translation * 10  # Apply translation and convert to mm
                final_translation_coordinate[opt_key][key] = random_translation * 10  # Store translation in mm
            
        return new_matrix, final_translation_coordinate


    def extract_details(self):
        examinations_transformations = []

        # Interested only in the examination part of the JSON file
        for exam_name, exam_details in self.data['examinations'].items():
            if 'pCT' not in exam_name:
                exam_dict = {'exam_name': exam_name, 'plans': []}  # Initialize with an empty plans list
                reg_to_planning = exam_details.get('registration_to_planning_examinations', {})

                for plan_name, registration_details in reg_to_planning.items():

                    # if 'F' not in plan_name:
                    if 'P' in plan_name:
                        from_exam = registration_details.get('from_examination_name')
                        to_exam = registration_details.get('to_examination_name')
                        rigid_matrix = registration_details.get('rigid_transformation_matrix', [])

                        if rigid_matrix:
                            new_matrix, final_translation_coordinate = self.new_transfer_param(rigid_matrix)
                            
                            plan_dict = {
                                'plan_name': plan_name,
                                'planning_exam_name': to_exam,
                                'repeatedCT_name': from_exam,
                                'transformation_matrix': new_matrix,
                                'final_translation_coordinate': final_translation_coordinate

                            }
                            exam_dict['plans'].append(plan_dict)  # Add this plan to the exam's plans list
                            print(plan_dict)
                if exam_dict['plans']:  # Only append if there are any plans
                    examinations_transformations.append(exam_dict)

        return examinations_transformations


    def get_plan_details(self):
        return self.examinations_transformations
    

    def save_to_json(self, output_file):
        data_to_save = {
        "id": self.id,
        "examination_details": self.examinations_transformations
    }
        with open(output_file, 'w') as file:
            json.dump(data_to_save, file, indent=4)

# # Usage example
# case_data_path = "/home/shahpouriz/Data/DBP_CTs/DBP_OP011_case_data.json"
# patient = CaseDataReader(case_data_path)

# examinations_transformations = patient.get_plan_details()
# patient.save_to_json('/home/shahpouriz/Data/DBP_CTs/DBP_OP011_coordination_data.json')


: 

In [ ]:
# Define the root directory to search for case_data.json files
case_data_root = "/home/shahpouriz/Data/TEST/"

# Use glob to find all *case_data.json files in the directory and its subdirectories
case_data_files = glob.glob(os.path.join(case_data_root, '**', '*case_data.json'), recursive=True)

for case_data_path in case_data_files:
    patient = CaseDataReader(case_data_path)

    examinations_transformations = patient.get_plan_details()
    
    # Generate the new file name based on the original file name
    # Here, we replace '_case_data.json' with '_coordination_data.json'
    new_file_name = case_data_path.replace('_case_data.json', '_coordination_data.json')
    
    # Optionally, ensure the directory exists or create it
    output_directory = os.path.dirname(new_file_name)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)
    
    # Save the processed data to the new file
    patient.save_to_json(new_file_name)


: 

In [ ]:
import warnings
import matplotlib.pyplot as plt
import SimpleITK as sitk

def process_examination_transformations(examinations_transformations, root_patient, saving_root_patient):
    """
    Process examination transformations for a given patient.

    Parameters:
    - examinations_transformations: List of examination transformations loaded from a JSON file.
    - root_patient: Base directory path where patient's DICOM files are located.
    - saving_root_patient: Directory path where processed data will be saved.
    """

    for item in examinations_transformations:
        exam_name = item.get('exam_name')
        for plan in item.get('plans', []):
            plan_name = plan.get('plan_name')
            moving_CT = plan.get('repeatedCT_name')
            fixed_CT = plan.get('planning_exam_name')

            for opt in ['opt1', 'opt2']:
                transformation_matrix = np.array(plan.get('transformation_matrix', {}).get(opt, [])).reshape(4, 4)

            
                print(f"Exam: {exam_name}, Plan: {plan_name}")
                print(f"Moving CT: {moving_CT}, Fixed (Planning) CT: {fixed_CT}")
                print(f"Transformation Matrix: {transformation_matrix}")
                print("--------------------------------------------------")
                
                path_fixed_CT = os.path.join(root_patient, fixed_CT, '')
                path_moving_CT = os.path.join(root_patient, moving_CT, '')

                if not os.path.exists(path_fixed_CT) or not os.path.exists(path_moving_CT):
                    warnings.warn(f"Path does not exist: {path_fixed_CT if not os.path.exists(path_fixed_CT) else path_moving_CT}")
                    continue  # Skip th

                path_struct_fixed_CT = get_path_structures_DICOM_file(path_fixed_CT)
                path_struct_moving_CT = get_path_structures_DICOM_file(path_moving_CT)

                fixed_CT_obj = construct_CT_object(fixed_CT, path_fixed_CT, path_struct_fixed_CT, roi_names=['External'])
                moving_CT_obj = construct_CT_object(moving_CT, path_moving_CT, path_struct_moving_CT, roi_names=['External'])

                
                moving_CT_obj.transform(transformation_matrix, fixed_CT_obj)
                

                fixed_CT_obj.override_air_outside_external()
                moving_CT_obj.override_air_outside_external()

                fixed_ct_saving_path = os.path.join(saving_root_patient, plan_name, opt, fixed_CT_obj.name)
                moving_ct_saving_path = os.path.join(saving_root_patient, plan_name, opt, moving_CT_obj.name)

                plan['planningCT_filename'][opt] = fixed_ct_saving_path
                plan['repeatedCT_filename'][opt] = moving_ct_saving_path


                # fixed_CT_array = sitk.GetArrayFromImage(fixed_CT_obj.image)
                # moving_CT_array = sitk.GetArrayFromImage(moving_CT_obj.image)
                # plt.imshow(fixed_CT_array[80]-moving_CT_array[80])
                
                if not os.path.exists(fixed_ct_saving_path):
                    os.makedirs(fixed_ct_saving_path, exist_ok=True)
                    fixed_CT_obj.save(fixed_ct_saving_path, save_struct_file=False)

                os.makedirs(moving_ct_saving_path, exist_ok=True)
                moving_CT_obj.save(moving_ct_saving_path, save_struct_file=False)

    

: 

In [ ]:
import glob
import os
import json
import numpy as np

# Define the directory where the coordination JSON files are saved
coordination_json_root = "/home/shahpouriz/Data/TEST/"

# Use glob to find all *_coordination_data.json files in the directory
coordination_json_files = glob.glob(os.path.join(coordination_json_root, '**', '*_coordination_data.json'), recursive=True)

for json_file in coordination_json_files:
    # Load the JSON content
    with open(json_file, 'r') as file:
        data = json.load(file)


    patient_id = data["id"]  # Assuming the patient ID is stored at the top level
    examinations_transformations = data["examination_details"]  # The list of transformations
    root_patient = os.path.join("/data/oosterhoff/patients/", patient_id)

    

    saving_root_patient = os.path.join(coordination_json_root, patient_id)

    process_examination_transformations(examinations_transformations, root_patient, saving_root_patient)


    with open(json_file, 'w') as file:
        json.dump(data, file, indent=4)


: 